In [ ]:
import os
import pandas as pd
import numpy as np
import anndata
import time
import matplotlib.pyplot as plt
import json
import requests
import pickle
import gzip as gz

import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src/')
from CellType_PSY import *

In [ ]:
def SaveDict(Dict, fname):
    with open(fname, 'wb') as hand:
        pickle.dump(Dict, hand)
    return

def LoadDict(fname):
    with open(fname, 'rb') as hand:
        b = pickle.load(hand)
        return b


# Make Cell Type Meta

In [ ]:
cell_extended = pd.read_csv("dat/cell_metadata_with_cluster_annotation.csv")

In [ ]:
label="class"
groupnames = cell_extended[label].unique()
group2barcode = {}
for groupname in groupnames:
    tmpdf = cell_extended[cell_extended[label]==groupname]
    group2barcode[groupname] = tmpdf["cell_barcode"].values

In [ ]:
groupnames.sort()
groupnames

In [ ]:
label="cluster"
groupnames = cell_extended[label].unique()
groupnames.sort()

In [ ]:
groupnames

In [ ]:
label="subclass"
groupnames = cell_extended[label].unique()
groupnames.sort()
groupnames

In [ ]:
cell_extended.head(5)

In [ ]:
feature_matrix_label = cell_extended["feature_matrix_label"].unique()
sub_meta_dict = {}
for feature_label in feature_matrix_label:
    sub_feature_meta = cell_extended[cell_extended["feature_matrix_label"]==feature_label]
    sub_meta_dict[feature_label] = sub_feature_meta
    sub_feature_meta.to_csv("dat/Cell_Meta_Splited/{}.meta.csv".format(feature_label))

In [ ]:
feature_matrix_label

In [ ]:
fillist = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/abc_download_root/expression_matrices/10X-SC-FileList.txt"

In [ ]:
flist = [l.strip() for l in open(fillist, 'rt').readlines()]

In [ ]:
import csv
fout = csv.writer(open("dat/ExpMat_Feature_Meta.tsv", 'wt'))
for expfil in flist:
    print(expfil)
    fname = expfil.split("/")[-1]
    feature_label = fname.replace("-log2.h5ad", "")
    if feature_label not in sub_meta_dict:
        print(feature_label)
    fout.writerow([expfil, feature_label, 
    "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Cell_Meta_Splited/{}.meta.csv".format(feature_label)])         

## Combine Feature seperated data (Mean)

In [ ]:
Split_Exp_Dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/AggregatedExpMat/"

In [ ]:
feature_matrix_label_v2 = [x for x in feature_matrix_label if 'v2' in x]
feature_matrix_label_v3 = [x for x in feature_matrix_label if 'v3' in x]
feature_matrix_label_Multi = [x for x in feature_matrix_label if 'Multi' in x]

In [ ]:
print(feature_matrix_label_v2)
print(feature_matrix_label_v3)
print(feature_matrix_label_Multi)

In [ ]:
# Make Expression Mat for v2, v3, multi seperetely

In [ ]:
Split_Exp_Dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/AggregatedExpMat/"

In [ ]:
#label = "subclass"
#label = "cluster"
label = "supertype"
Splited_Mats_v2 = {}
NumCells_v2 = {}
for feature in feature_matrix_label_v2:
    SubExpMat = pd.read_csv("{}/ExpMat.{}.{}.csv".format(Split_Exp_Dir, feature, label), index_col=0)
    CellNum = LoadDict("{}/NumCells.{}.{}.csv".format(Split_Exp_Dir, feature, label))
    Splited_Mats_v2[feature] = SubExpMat
    NumCells_v2[feature] = CellNum

In [ ]:
Splited_Mats_v3 = {}
NumCells_v3 = {}
for feature in feature_matrix_label_v3:
    SubExpMat = pd.read_csv("{}/ExpMat.{}.{}.csv".format(Split_Exp_Dir, feature, label), index_col=0)
    CellNum = LoadDict("{}/NumCells.{}.{}.csv".format(Split_Exp_Dir, feature, label))
    Splited_Mats_v3[feature] = SubExpMat
    NumCells_v3[feature] = CellNum

In [ ]:
Splited_Mats_multi = {}
NumCells_multi = {}
for feature in feature_matrix_label_Multi:
    SubExpMat = pd.read_csv("{}/ExpMat.{}.{}.csv".format(Split_Exp_Dir, feature, label), index_col=0)
    CellNum = LoadDict("{}/NumCells.{}.{}.csv".format(Split_Exp_Dir, feature, label))
    Splited_Mats_multi[feature] = SubExpMat
    NumCells_multi[feature] = CellNum

In [ ]:
CellTypes = cell_extended[label].unique()
CellTypes.sort()

In [ ]:
Genes = Splited_Mats_v2[feature_matrix_label_v2[0]].index.values

In [ ]:
CombineMat_v2 = pd.DataFrame(data=np.zeros((len(Genes), len(CellTypes))), index=Genes, columns=CellTypes)
TotalCellNum_v2 = np.zeros(len(CellTypes))
for feature in feature_matrix_label_v2:
    SubExpMat = Splited_Mats_v2[feature]
    CellNum = NumCells_v2[feature] 
    for i, CT in enumerate(CellTypes):
        if CT not in CellNum:
            addCol = np.zeros(len(Genes))
        else:
            addCol = SubExpMat[CT].values * CellNum[CT]
            TotalCellNum_v2[i] = TotalCellNum_v2[i] + CellNum[CT]
        CombineMat_v2[CT] = CombineMat_v2[CT] + addCol
for i, CT in enumerate(CellTypes):
    if not TotalCellNum_v2[i] == 0:  
        CombineMat_v2[CT] = CombineMat_v2[CT]/TotalCellNum_v2[i]

In [ ]:
CombineMat_v3 = pd.DataFrame(data=np.zeros((len(Genes), len(CellTypes))), index=Genes, columns=CellTypes)
TotalCellNum_v3 = np.zeros(len(CellTypes))
for feature in feature_matrix_label_v3:
    SubExpMat = Splited_Mats_v3[feature]
    CellNum = NumCells_v3[feature] 
    for i, CT in enumerate(CellTypes):
        if CT not in CellNum:
            addCol = np.zeros(len(Genes))
        else:
            addCol = SubExpMat[CT].values * CellNum[CT]
            TotalCellNum_v3[i] = TotalCellNum_v3[i] + CellNum[CT]
        CombineMat_v3[CT] = CombineMat_v3[CT] + addCol
for i, CT in enumerate(CellTypes):
    if not TotalCellNum_v3[i] == 0:  
        CombineMat_v3[CT] = CombineMat_v3[CT]/TotalCellNum_v3[i]

In [ ]:
print("Done Part")

#### ## Combine Feature seperated data (Median)

In [ ]:
Split_Exp_Dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/AggregatedExpMat_Medium/"

In [ ]:
feature_matrix_label_v3 = [x for x in feature_matrix_label if 'v3' in x]
print(feature_matrix_label_v3)

In [ ]:
#label = "supertype"
label = "subclass"
Splited_Mats_v3 = {}
NumCells_v3 = {}
for feature in feature_matrix_label_v3:
    SubExpMat = pd.read_csv("{}/ExpMat.{}.{}.csv".format(Split_Exp_Dir, feature, label), index_col=0)
    CellNum = LoadDict("{}/NumCells.{}.{}.csv".format(Split_Exp_Dir, feature, label))
    print(SubExpMat.shape)
    Splited_Mats_v3[feature] = SubExpMat
    NumCells_v3[feature] = CellNum

In [ ]:
CellTypes = cell_extended[label].unique()
CellTypes.sort()

In [ ]:
Genes = Splited_Mats_v3[feature_matrix_label_v3[0]].index.values

In [ ]:
CombineMat_v3 = pd.DataFrame(data=np.zeros((len(Genes), len(CellTypes))), index=Genes, columns=CellTypes)
for g in Genes:
    for i, CT in enumerate(CellTypes):
        dat = []
        for feature in feature_matrix_label_v3:
            SubExpMat = Splited_Mats_v3[feature]
            CellNum = NumCells_v3[feature] 
            if CT in CellNum:
                dat.extend([SubExpMat.loc[g, CT]]*CellNum[CT])
        if len(dat) > 0:
            median = np.median(dat)
        else:
            median = 0
        CombineMat_v3.loc[g, CT] = median

In [ ]:
Subclass_V3_ExpMat = ExpMatConvertEntrez(CombineMat_v3, ENSMUSG2HumanEntrez)
Subclass_V3_ExpMat.to_csv("dat/ExpressionMats_Median/{}_V3_ExpMat.csv".format(label))

#### Compare Mean and Median

In [ ]:
MeanDF = pd.read_csv("dat/ExpressionMats/Subclass_V3_ExpMat.csv", index_col=0)
MedianDF = pd.read_csv("dat/ExpressionMats_Median/subclass_V3_ExpMat.csv", index_col=0)

In [ ]:
MeanDF.head(2)

In [ ]:
MedianDF.head(2)

In [ ]:
CT1 = "001 CLA-EPd-CTX Car3 Glut"
X = MeanDF[CT1].values
Y = MedianDF[CT1].values

plt.figure(dpi=150, figsize=(8, 6))

# Scatter plot
plt.scatter(X, Y, color='blue', alpha=0.7, label='Data points')

# Adding the X=Y line
min_val = min(min(X), min(Y))
max_val = max(max(X), max(Y))
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', label='X=Y line')

# Labels and title
plt.xlabel('Mean', fontsize=14)
plt.ylabel('Median', fontsize=14)
plt.title(f'Scatter Plot of Mean vs Median for {CT}', fontsize=16)

# Adding a legend
plt.legend(fontsize=12)

# Adding grid lines
plt.grid(True, linestyle='--', alpha=0.6)

# Adjusting tick parameters for better readability
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Setting equal aspect ratio for x and y axis
plt.gca().set_aspect('equal', adjustable='box')

# Showing the plot
plt.tight_layout()
plt.show()

In [ ]:
CT2 = "123 DMH Nkx2-4 Glut"
X = MeanDF[CT2].values
Y = MedianDF[CT2].values

plt.figure(dpi=150, figsize=(8, 6))

# Scatter plot
plt.scatter(X, Y, color='blue', alpha=0.7, label='Data points')

# Adding the X=Y line
min_val = min(min(X), min(Y))
max_val = max(max(X), max(Y))
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', label='X=Y line')

# Labels and title
plt.xlabel('Mean', fontsize=14)
plt.ylabel('Median', fontsize=14)
plt.title(f'Scatter Plot of Mean vs Median for {CT}', fontsize=16)

# Adding a legend
plt.legend(fontsize=12)

# Adding grid lines
plt.grid(True, linestyle='--', alpha=0.6)

# Adjusting tick parameters for better readability
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Setting equal aspect ratio for x and y axis
plt.gca().set_aspect('equal', adjustable='box')

# Showing the plot
plt.tight_layout()
plt.show()

In [ ]:
print(CT1)
print(CT2)

In [ ]:
X = MeanDF[CT1].values
Y = MeanDF[CT2].values

plt.figure(dpi=150, figsize=(8, 6))

# Scatter plot
plt.scatter(X, Y, color='blue', alpha=0.7, label='Data points')

# Adding the X=Y line
min_val = min(min(X), min(Y))
max_val = max(max(X), max(Y))
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', label='X=Y line')

# Labels and title
plt.xlabel('Mean', fontsize=14)
plt.ylabel('Median', fontsize=14)
plt.title(f'Scatter Plot of Mean vs Median for {CT}', fontsize=16)

# Adding a legend
plt.legend(fontsize=12)

# Adding grid lines
plt.grid(True, linestyle='--', alpha=0.6)

# Adjusting tick parameters for better readability
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Setting equal aspect ratio for x and y axis
plt.gca().set_aspect('equal', adjustable='box')

# Showing the plot
plt.tight_layout()
plt.show()

In [ ]:
X = MedianDF[CT1].values
Y = MedianDF[CT2].values

plt.figure(dpi=150, figsize=(8, 6))

# Scatter plot
plt.scatter(X, Y, color='blue', alpha=0.7, label='Data points')

# Adding the X=Y line
min_val = min(min(X), min(Y))
max_val = max(max(X), max(Y))
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', label='X=Y line')

# Labels and title
plt.xlabel('Mean', fontsize=14)
plt.ylabel('Median', fontsize=14)
plt.title(f'Scatter Plot of Mean vs Median for {CT}', fontsize=16)

# Adding a legend
plt.legend(fontsize=12)

# Adding grid lines
plt.grid(True, linestyle='--', alpha=0.6)

# Adjusting tick parameters for better readability
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Setting equal aspect ratio for x and y axis
plt.gca().set_aspect('equal', adjustable='box')

# Showing the plot
plt.tight_layout()
plt.show()

In [ ]:
Cluster_V3_ExpMat2 = pd.read_csv("dat/ExpressionMats_Median/test.cluster_V3_ExpMat.csv", index_col=0)

In [ ]:
Cluster_V3_ExpMat2_HumanGene = ExpMatConvertEntrez(Cluster_V3_ExpMat2, ENSMUSG2HumanEntrez)
Cluster_V3_ExpMat2_HumanGene.to_csv("dat/ExpressionMats_Median/Cluster_V3_ExpMat.csv")

In [ ]:
Cluster_V3_ExpMat2_HumanGene.head(2)

In [ ]:
SuperType_V3_ExpMat2 = pd.read_csv("dat/ExpressionMats_Median/test.supertype_V3_ExpMat.csv", index_col=0)
SuperType_V3_ExpMat2_HumanGene = ExpMatConvertEntrez(SuperType_V3_ExpMat2, ENSMUSG2HumanEntrez)
SuperType_V3_ExpMat2_HumanGene.to_csv("dat/ExpressionMats_Median/Supertype_V3_ExpMat.csv")

In [ ]:
STR_MERFISH_MarkerG = pd.read_csv("dat/MERFISH_MarkerG/test.csv", index_col=0)
STR_MERFISH_MarkerG_HumanGene = ExpMatConvertEntrez(STR_MERFISH_MarkerG, ENSMUSG2HumanEntrez)
STR_MERFISH_MarkerG_HumanGene.to_csv("dat/MERFISH_MarkerG/STR_MERFISH_MarkerG.csv")

#### [end] Compare Mean and Median

###### Old

In [ ]:
Split_Exp_Dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/AggregatedExpMat/"

In [ ]:
#label = "subclass"
label = "cluster"
Splited_Mats = {}
NumCells = {}
for feature in feature_matrix_label:
    SubExpMat = pd.read_csv("{}/ExpMat.{}.{}.csv".format(Split_Exp_Dir, feature, label), index_col=0)
    CellNum = LoadDict("{}/NumCells.{}.{}.csv".format(Split_Exp_Dir, feature, label))
    Splited_Mats[feature] = SubExpMat
    NumCells[feature] = CellNum

In [ ]:
CellTypes = cell_extended[label].unique()
CellTypes.sort()

In [ ]:
Genes = Splited_Mats[feature_matrix_label[0]].index.values

In [ ]:
CombineMat = pd.DataFrame(data=np.zeros((len(Genes), len(CellTypes))), index=Genes, columns=CellTypes)

In [ ]:
CombineMat

In [ ]:
TotalCellNum = np.zeros(len(CellTypes))
for feature in feature_matrix_label:
    SubExpMat = Splited_Mats[feature]
    CellNum = NumCells[feature] 
    for i, CT in enumerate(CellTypes):
        if CT not in CellNum:
            addCol = np.zeros(len(Genes))
        else:
            addCol = SubExpMat[CT].values * CellNum[CT]
            TotalCellNum[i] = TotalCellNum[i] + CellNum[CT]
        CombineMat[CT] = CombineMat[CT] + addCol

In [ ]:
for i, CT in enumerate(CellTypes):
    CombineMat[CT] = CombineMat[CT]/TotalCellNum[i]

In [ ]:
CombineMat

#### convert index to humen entrez ID

###### mouse ENSMUSG ID to symbol

In [ ]:
def parse_gtf(file_path):
    features = []
    GeneID2GeneSymbol = {}
    Failed_Mapping = []
    with gz.open(file_path, 'rt') as gtf_file:
        for line in gtf_file:
            if line.startswith('#'):
                continue  # Skip comment lines
            fields = line.strip().split('\t')
            # GTF fields: seqname, source, feature, start, end, score, strand, frame, attributes
            seqname, source, feature, start, end, score, strand, frame, attributes = fields
            # Only consider gene feature
            if feature != "gene":
                continue
            # Parse attributes into a dictionary
            attribute_dict = {}
            for attribute in attributes.split('; '):
                #print(attribute)
                key, value = attribute.strip().split(' ')
                attribute_dict[key] = value.strip('"').strip('";"')
            # Get gene_id to symnol map
            if attribute_dict["gene_biotype"] != "protein_coding":
                #print(attribute_dict["gene_biotype"])
                continue
            try:
                GeneID2GeneSymbol[attribute_dict["gene_id"]] = attribute_dict["gene_name"]
            except:
                Failed_Mapping.append(attribute_dict)
    return GeneID2GeneSymbol

In [ ]:
gtf_file_path = '/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/Mus_musculus.GRCm39.110.gtf.gz'
Mouse_GeneID2GeneSymbol = parse_gtf(gtf_file_path)

In [ ]:
print(len(Mouse_GeneID2GeneSymbol))

###### Human 2 mouse gene mapping

In [ ]:
Human2MouseHom = pd.read_csv("../../ASD_Circuits/scripts//HOM_MouseHumanSequence.rpt", delimiter="\t")

In [ ]:
Homo_IDs = set(Human2MouseHom["DB Class Key"].values)
print(len(Homo_IDs))

In [ ]:
MouseSymbol2HumanEntrez = {}
Human2Mouse_Genes = {}
Mouse2Human_Genes = {}
for counter, ID in enumerate(Homo_IDs):
    tmp_df = Human2MouseHom[Human2MouseHom["DB Class Key"]==ID]

    hum_genes, mou_genes = [], []
    for i, row in tmp_df.iterrows():
        TaxonID = row["NCBI Taxon ID"]
        Symbol = row["Symbol"]
        Entrez = row["EntrezGene ID"]
        if  TaxonID == 9606:
            hum_genes.append((Symbol, Entrez))
        elif TaxonID == 10090:
            mou_genes.append((Symbol, Entrez))
    # Fill up Human2Mouse_Genes
    for (Symbol, Entrez) in hum_genes:
        if Entrez not in Human2Mouse_Genes:
            Human2Mouse_Genes[Entrez] = {}#mou_genes
            Human2Mouse_Genes[Entrez]["symbol"] = Symbol
            Human2Mouse_Genes[Entrez]["mouseHomo"] = mou_genes
        else:
            Human2Mouse_Genes[Entrez]["mouseHomo"].extend(mou_genes)
        
    # Fill up Mouse2Human_Genes
    for (Symbol, Entrez) in mou_genes:
        if Entrez not in Mouse2Human_Genes:
            Mouse2Human_Genes[Symbol] = {} #mou_genes
            Mouse2Human_Genes[Symbol]["entrez"] = Entrez
            Mouse2Human_Genes[Symbol]["humanHomo"] = hum_genes
        else:
            Mouse2Human_Genes[Symbol]["humanHomo"].extend(hum_genes)

In [ ]:
Mouse2Human_Genes

In [ ]:
with open("Mouse2HumanGenes", 'w') as json_file:
    json.dump(Mouse2Human_Genes, json_file)

##### Check mapping from ABC atlas to Human Entrez 

In [ ]:
def ExpMatConvertEntrez(CombineMat, ENSMUSG2HumanEntrez):
    dat_rows = []
    index = []
    for ensg, row in CombineMat.iterrows():
        if ensg in ENSMUSG2HumanEntrez:
            for entrez in ENSMUSG2HumanEntrez[ensg]:
                index.append(entrez)
                dat_rows.append(row)
    ExpDF_humanEntrez = pd.DataFrame(data=dat_rows, index=index, columns=CombineMat.columns.values)
    ExpDF_humanEntrezDedup = ExpDF_humanEntrez[~ExpDF_humanEntrez.index.duplicated(keep='first')]
    return ExpDF_humanEntrezDedup

In [ ]:
Subclass_V2_ExpMat = ExpMatConvertEntrez(CombineMat_v2, ENSMUSG2HumanEntrez)
Subclass_V3_ExpMat = ExpMatConvertEntrez(CombineMat_v3, ENSMUSG2HumanEntrez)

In [ ]:
Subclass_V2_ExpMat.to_csv("dat/ExpressionMats/{}_V2_ExpMat.csv".format(label))
Subclass_V3_ExpMat.to_csv("dat/ExpressionMats/{}_V3_ExpMat.csv".format(label))

In [ ]:
#ENSMUSG2HumanEntrez.to
with open("Mouse2HumanGenes", 'w') as json_file:
    json.dump(Mouse2Human_Genes, json_file)

In [ ]:
ENSMUSG2HumanEntrez
with open("dat/ENSMUSG2HumanEntrez.json", 'w') as json_file:
    json.dump(ENSMUSG2HumanEntrez, json_file)

###### Old

In [ ]:
ENSMUSG_no_symbol_or_not_protein_coding = []
Mouse2Human_noMap = []
Mouse2Human_uniqMap = []
Mouse2Human_multiMap = []
ENSMUSG2HumanEntrez = {}
for msg in Genes:
    if msg in Mouse_GeneID2GeneSymbol:
        mouse_symbol = Mouse_GeneID2GeneSymbol[msg]
        if mouse_symbol not in Mouse2Human_Genes:
            Mouse2Human_noMap.append(msg)
            continue
        human_entrez = Mouse2Human_Genes[mouse_symbol]["humanHomo"]
        if len(human_entrez) == 0:
            Mouse2Human_noMap.append(msg)
        if len(human_entrez) == 1:
            Mouse2Human_uniqMap.append(msg)
            ENSMUSG2HumanEntrez[msg] = [x[1] for x in human_entrez]
        else:
            Mouse2Human_multiMap.append(msg)
            ENSMUSG2HumanEntrez[msg] = [x[1] for x in human_entrez]
    else:
        ENSMUSG_no_symbol_or_not_protein_coding.append(msg)

In [ ]:
len(ENSMUSG2HumanEntrez)

In [ ]:
print(len(ENSMUSG_no_symbol_or_not_protein_coding))
print(len(Mouse2Human_noMap))
print(len(Mouse2Human_uniqMap))
print(len(Mouse2Human_multiMap))

In [ ]:
def AvgDupes(df):
    duplicate_index = df[df.index.duplicated(keep=False)]

    # Iterate through duplicate index values and average their data
    for index_value in duplicate_index.index.unique():
        duplicate_rows = df.loc[df.index == index_value]
        average_values = duplicate_rows.mean(axis=0)
    # Drop the duplicated index values
    df = df[~df.index.duplicated(keep='first')]
    df.loc[index_value, :] = average_values
    return df

In [ ]:
dat_rows = []
index = []
for ensg, row in CombineMat.iterrows():
    if ensg in ENSMUSG2HumanEntrez:
        for entrez in ENSMUSG2HumanEntrez[ensg]:
            index.append(entrez)
            dat_rows.append(row)
ExpDF_humanEntrez = pd.DataFrame(data=dat_rows, index=index, columns=CombineMat.columns.values)

In [ ]:
#ExpDF_humanEntrezDedup = AvgDupes(ExpDF_humanEntrez)
ExpDF_humanEntrezDedup = ExpDF_humanEntrez[~ExpDF_humanEntrez.index.duplicated(keep='first')]
ExpDF_humanEntrezDedup.to_csv("dat/ExpMat.{}.HumanEntrez.csv".format(label))

In [ ]:
ExpDF_humanEntrezDedup

In [ ]:
print(len(Mouse2Human_uniqMap))
print(len(ENSMUSG2HumanEntrez))

#### Cal Z1 Mat

In [ ]:
def Z1Conversion(ExpMat, outname="test.z1.mat"):
    Z_mat = []
    for g, row in ExpMat.iterrows():
        tmp = ZscoreConverting(row.values)
        Z_mat.append(tmp)
    Z_mat = np.array(Z_mat)
    CT_Z1_DF = pd.DataFrame(data=Z_mat, index=ExpMat.index.values, 
                            columns=ExpMat.columns.values)
    CT_Z1_DF.to_csv(outname)
    return CT_Z1_DF

In [ ]:
MedianDF = pd.read_csv("dat/ExpressionMats_Median/subclass_V3_ExpMat.csv", index_col=0)
MedianDF_Z1 = Z1Conversion(MedianDF, "dat/ExpressionMats_Median/Subclass_V3_Z1Mat.csv")

In [ ]:
Z1Conversion(Subclass_V2_ExpMat, "dat/ExpressionMats/Subclass_V2_Z1Mat.csv")
Z1Conversion(Subclass_V3_ExpMat, "dat/ExpressionMats/Subclass_V3_Z1Mat.csv")

In [ ]:
Cluster_CB_ExpMat = pd.read_csv("dat/ExpressionMats/Cluster_CB_ExpMat.csv", index_col=0)
Z1Conversion(Cluster_CB_ExpMat, "dat/ExpressionMats/Cluster_CB_Z1Mat.csv")

In [ ]:
Cluster_CB_ExpMat

In [ ]:
Z_mat = []
for g, row in ExpDF_humanEntrezDedup.iterrows():
    tmp = ZscoreConverting(row.values)
    Z_mat.append(tmp)
Z_mat = np.array(Z_mat)
CT_Z1_DF = pd.DataFrame(data=Z_mat, index=ExpDF_humanEntrezDedup.index.values, 
                        columns=ExpDF_humanEntrezDedup.columns.values)
CT_Z1_DF.to_csv("dat/Z1Mat.{}.HumanEntrez.csv".format(label))

In [ ]:
FusionExpMat = pd.read_csv("dat/ExpressionMats/Fusion_Exp.csv", index_col=0)
Z1Conversion(FusionExpMat, "dat/ExpressionMats/Fusion_Z1Mat.csv")

In [ ]:
FusionExpMat = pd.read_csv("dat/ExpressionMats_Median/Fusion_Exp.csv", index_col=0)
FusionZ1 = Z1Conversion(FusionExpMat, "dat/ExpressionMats_Median//Fusion_Z1Mat.csv")

In [ ]:
ExpMat = pd.read_csv("dat/MERFISH_Zhuang/STR_Cell_Mean_DF.csv", index_col=0)
Z1Mat = Z1Conversion(ExpMat, "dat/MERFISH_Zhuang/STR_Cell_Mean_DF.Z1.csv")
ExpMat = pd.read_csv("dat/MERFISH_Zhuang/STR_Vol_Mean_DF.csv", index_col=0)
Z1Mat = Z1Conversion(ExpMat, "dat/MERFISH_Zhuang/STR_Vol_Mean_DF.Z1.csv")

##### Cal Exp Matching features

In [ ]:
g_exp_v2 = []
for i, g in enumerate(Subclass_V2_ExpMat.index.values):
    res = 0
    for j, c in enumerate(Subclass_V2_ExpMat.columns.values):
        exp_l = Subclass_V2_ExpMat.loc[g, c]
        if exp_l == exp_l:
            res += exp_l * TotalCellNum_v2[j]
        else:
            res += 0
    g_exp_v2.append(res)
    if i % 100 ==0:
        print("processed",i,"genes") 

In [ ]:
V2_Exp_DF = pd.DataFrame(data={"Gene":Subclass_V2_ExpMat.index.values, "Exp":g_exp_v2})
V2_Exp_DF = V2_Exp_DF.sort_values("Exp")
V2_Exp_DF["Rank"] = [1+x for x in range(V2_Exp_DF.shape[0])] # compute Rank
V2_Exp_DF["quantile"] = V2_Exp_DF["Rank"]/V2_Exp_DF.shape[0]
V2_Exp_DF = V2_Exp_DF.set_index("Gene")
V2_Exp_DF.to_csv("dat/ABC_TotalExp.Match.10xV2.csv")

In [ ]:
g_exp_v3 = []
for i, g in enumerate(Subclass_V3_ExpMat.index.values):
    res = 0
    for j, c in enumerate(Subclass_V3_ExpMat.columns.values):
        exp_l = Subclass_V2_ExpMat.loc[g, c]
        if exp_l == exp_l:
            res += exp_l * TotalCellNum_v3[j]
        else:
            res += 0
    g_exp_v3.append(res)
    if i % 100 ==0:
        print("processed",i,"genes") 

In [ ]:
V3_Exp_DF = pd.DataFrame(data={"Gene":Subclass_V3_ExpMat.index.values, "Exp":g_exp_v3})
V3_Exp_DF = V3_Exp_DF.sort_values("Exp")
V3_Exp_DF["Rank"] = [1+x for x in range(V3_Exp_DF.shape[0])] # compute Rank
V3_Exp_DF["quantile"] = V3_Exp_DF["Rank"]/V3_Exp_DF.shape[0]
V3_Exp_DF = V3_Exp_DF.set_index("Gene")
V3_Exp_DF.to_csv("dat/ABC_TotalExp.Match.10xV3.csv")

In [ ]:
g_exp_wb = []
for i, g in enumerate(ExpDF_humanEntrezDedup.index.values):
    res = 0
    for j, c in enumerate(ExpDF_humanEntrezDedup.columns.values):
        exp_l = ExpDF_humanEntrezDedup.loc[g, c]
        if exp_l == exp_l:
            res += exp_l * TotalCellNum[j]
        else:
            res += 0
    g_exp_wb.append(res)
    if i % 100 ==0:
        print("processed",i,"genes") 

In [ ]:
g_exp_wb

In [ ]:
Neuro_Exp_DF = pd.DataFrame(data={"Gene":ExpDF_humanEntrezDedup.index.values, "Exp":g_exp_wb})
Neuro_Exp_DF = Neuro_Exp_DF.sort_values("Exp")
Neuro_Exp_DF["Rank"] = [1+x for x in range(Neuro_Exp_DF.shape[0])] # compute Rank
Neuro_Exp_DF["quantile"] = Neuro_Exp_DF["Rank"]/Neuro_Exp_DF.shape[0]
Neuro_Exp_DF = Neuro_Exp_DF.drop_duplicates(subset="Gene", keep="first")
Neuro_Exp_DF = Neuro_Exp_DF.set_index("Gene")
Neuro_Exp_DF.to_csv("dat/ABC_TotalExp.Match.v2.csv")

In [ ]:
Neuro_Exp_DF

### Combine Z2 Mat

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/subclass_AllenRegionMatch/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.subclass.HumanEntrez.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/subclass_ABC_TotalExp_Match/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.subclass.HumanEntrez.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/subclass_ABC_TotalExp_Match_Exp/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch_Exp.subclass.HumanEntrez.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/cluster_ABC_TotalExp_Match_Exp/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch_Exp.cluster.HumanEntrez.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/cluster_ABC_TotalExp_Match/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.cluster.HumanEntrez.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Subclass_V2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.subclass.10xV2.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Subclass_V3/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.subclass.10xV3.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Cluster_V2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.cluster.10xV2.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Cluster_V3/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.cluster.10xV3.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Cluster_CB//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.cluster.CB.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Fusion.v2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.cluster.Fusion.V2.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Fusion_Zmatch_v3//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.cluster.Fusion.Zmatch.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Median_Fusion/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.Median_Fusion.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Subclass_median_V3/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.Subclass_MedianV3.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_CellMean_V3_Zmatch_SCMatch/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH/Z2Mat.ABCMatch.ISH_STR.CellMean.Zmatch.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_VolMean_V3_Zmatch_SCMatch/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH/Z2Mat.ABCMatch.ISH_STR.VolMean.Zmatch.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_median_CellMean_V3/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH_Median/Z2Mat.ABCMatch.ISH_STR.MedianV3.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_median_CellMean_V3_SCMatch//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH_Median/Z2Mat.ABCMatch.ISH_STR.MedianV3.CellMean.SCMatch.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_median_VolMean_V3_SCMatch/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH_Median/Z2Mat.ABCMatch.ISH_STR.MedianV3.VolMean.SCMatch.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_Zhuang_CB_CellMean_V3_SCMatch//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH_Zhuang/Z2Mat.ISH_STR.CellMean.SCMatch.csv")

Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_Zhuang_CB_VolMean_V3_SCMatch//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH_Zhuang/Z2Mat.ISH_STR.VolMean.SCMatch.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_Zhuang_CB_CellMean_V3_SCMatch_Zmatch//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH_Zhuang/Z2Mat.ISH_STR.CellMean.SCMatch.Zmatch.csv")

Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_Zhuang_CB_VolMean_V3_SCMatch_Zmatch//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH_Zhuang/Z2Mat.ISH_STR.VolMean.SCMatch.Zmatch.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MouseISH_ISHMatch_Zmatch/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH/Z2Mat.ISH.Zmatch.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MouseCT_Cluster_V3_Zmatch/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/ExpressionMats/MouseCT.Cluster.10v3.Zmatch.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MouseCT_Subclass_V2_Zmatch//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/ExpressionMats/MouseCT.Subclass.10v2.Zmatch.csv")

Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MouseCT_Subclass_V3_Zmatch/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/ExpressionMats/MouseCT.Subclass.10v3.Zmatch.csv")

In [ ]:
submat_L4_5IT_CTX_Glut_6_entrez
submat_TH_Prkcd_Glut_10_entrez

## Try QuntileNorm

In [ ]:
Exp_Subclass_CB = pd.read_csv("dat/ExpressionMats/Subclass_CB_ExpMat.csv", index_col=0)
#Exp_Subclass_CB = #Exp_Subclass_CB.fillna(0)

In [ ]:
Exp_Subclass_CB2 = Exp_Subclass_CB.dropna(axis=1, how='all')
Exp_Subclass_CB2.shape

In [ ]:
Subclass_CB_Z1 = Z1Conversion(Exp_Subclass_CB2, "dat/ExpressionMats/Subclass_CB_Z1Mat.csv")

In [ ]:
Exp_Subclass_CB_qn = quantileNormalize(Exp_Subclass_CB)

In [ ]:
Subclass_CB_Z1 = Z1Conversion(Exp_Subclass_CB, "dat/ExpressionMats/Subclass_CB_Z1Mat.csv")
Subclass_CB_qn_Z1 = Z1Conversion(Exp_Subclass_CB_qn, "dat/ExpressionMats/Subclass_CB_qn_Z1Mat.csv")
Subclass_CB_qn_Z1_qn = quantileNormalize(Subclass_CB_qn_Z1)
Subclass_CB_qn_Z1_qn.to_csv("dat/ExpressionMats/Subclass_CB_qn_Z1Mat_qn.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Subclass_CB/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.Subclass.CB.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Subclass_CB_qn/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.Subclass.CB.qn.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Subclass_CB_qn_qn/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.Subclass.CB.qn.qn,csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Test.MNSP/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/SpecMat.ABCMatch.Subclass.CB.csv")
Z2_MAt.shape

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Test.MNSP.2//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/SpecMat.ABCMatch.Subclass.CB.2.csv")
Z2_MAt.shape

### Increase Isocortex cell type and decrease Hindbrain cell types

In [ ]:
SubclassAnn = pd.read_excel("../../data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx",
                          sheet_name = "subclass_annotation", index_col="subclass_id_label")
ClusterAnn = pd.read_excel("../../data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx",
                          sheet_name = "cluster_annotation", index_col="cluster_id_label")

In [ ]:
ClusterLevel = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut', '04 DG-IMN Glut', '05 OB-IMN GABA',
    '06 CTX-CGE GABA', '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA', '10 LSX GABA', ]
SubclassLevel = ['11 CNU-HYa GABA', '12 HY GABA', '13 CNU-HYa Glut', '14 HY Glut', '15 HY Gnrh1 Glut', 
    '16 HY MM Glut', '17 MH-LH Glut', '18 TH Glut', '19 MB Glut', '20 MB GABA', '21 MB Dopa', '22 MB-HB Sero', 
    '23 P Glut', '24 MY Glut', '25 Pineal Glut', '26 P GABA', '27 MY GABA', '28 CB GABA', '29 CB Glut',
    '30 Astro-Epen', '31 OPC-Oligo', '32 OEC', '33 Vascular', '34 Immune']

In [ ]:
X_Clusters = ClusterAnn[ClusterAnn["class_id_label"].isin(ClusterLevel)].index.values
X_Subclass = SubclassAnn[SubclassAnn["class_id_label"].isin(SubclassLevel)].index.values

In [ ]:
print(len(X_Clusters), len(X_Subclass))

In [ ]:
Subclass_Exp = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/ExpressionMats/Subclass_CB_ExpMat.csv", index_col=0)
Cluster_Exp  = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/ExpressionMats/Cluster_CB_ExpMat.csv", index_col=0)

In [ ]:
Sub_Cluster_Exp = Cluster_Exp.loc[:, X_Clusters]
Sub_Subclass_Exp = Subclass_Exp.loc[:, X_Subclass]

In [ ]:
print(Sub_Cluster_Exp.shape, Sub_Subclass_Exp.shape)

In [ ]:
Fusion_Exp = pd.concat([Sub_Cluster_Exp, Sub_Subclass_Exp], axis=1)

In [ ]:
Fusion_Exp.shape

In [ ]:
Z1Conversion(Fusion_Exp, "dat/ExpressionMats/Fusion_Exp_Z1Mat.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/Fusion_CB/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/Z2Mat.ABCMatch.Fusion.CB.csv")
Z2_MAt.shape

### Make Cell Type hierarchy

In [ ]:
Class = sorted(cell_extended["class"].unique())
subclass = sorted(cell_extended["subclass"].unique())
supertype = sorted(cell_extended["supertype"].unique())
clusters = sorted(cell_extended["cluster"].unique())

In [ ]:
Class

In [ ]:
CellTypesDF = cell_extended.set_index("cluster")
CellTypesDF = CellTypesDF[~CellTypesDF.index.duplicated(keep='first')]
CellTypesDF = CellTypesDF[["class", "subclass", "supertype"]]

In [ ]:
CellTypesDF.to_csv("dat/CellTypeHierarchy.csv")

In [ ]:
#for cluster, row in CellTypesDF.iterrows():

# Subclass 

In [ ]:
ClusterAnn = pd.read_excel("/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx", sheet_name = "cluster_annotation", index_col="cluster_id_label")

DatDIR = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/Cluster_GeneXCell_UMI/"
Subclasses = ClusterAnn["subclass_id_label"].unique()
print(Subclasses)

In [ ]:
results = []
input_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/Subclass_SplitCTs/"
# read files in input_dir
No_Data_Subclasses = []
for subclass in Subclasses:
    subclass_clean_name = re.sub(r'\W+', '_', subclass)
    try:
        results.append(pd.read_csv(input_dir + "{}.csv".format(subclass_clean_name), index_col=0))
    except FileNotFoundError:
        print("File not found for subclass: {}".format(subclass))
        print("File Name is: {}".format(input_dir + "{}.csv".format(subclass_clean_name)))
        #results.append(pd.DataFrame(0, index=range(17938), columns=["0", "1", "2" ]))
        No_Data_Subclasses.append(subclass)


In [ ]:
res

In [ ]:
Indv_subclass_means = []
Indv_subclass_total_UMIs = []
#Gene_Total_Exp = np.zeros(len(results[0][1]))
Total_N_Cells = 0
for subclass, res in zip(Subclasses, results):
    #print(subclass, res)
    if subclass in No_Data_Subclasses:
        continue
    gene_mean_logUMI = res["0"]
    gene_mean_logUMI.name = subclass

    Indv_subclass_means.append(gene_mean_logUMI)

# Make and save cluster Exp Mat
Subclass_Exp_DF = pd.concat(Indv_subclass_means, axis=1)
Subclass_Exp_DF.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/subclass_MeanLogUMI.csv")


In [ ]:
Subclass_Exp_DF

In [ ]:
idx = Subclass_Exp_DF.index.values[9]
print(idx)
Subclass_Exp_DF.loc[idx, :].hist()

In [ ]:
Subclass_Exp_DF = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/subclass_MeanLogUMI.csv", index_col=0)


In [ ]:
Subclass_Z1 = Z1Conversion(Subclass_Exp_DF, "dat/SC_UMI_Mats/Subclass_UMI_Z1Mat.csv")

In [ ]:
def ZscoreConverting_V2(values, mean=np.nan, std=np.nan, low_exp = 0, min_z=-5): 
    """
    Convert values to z-scores with special handling for zeros:
    - Build distribution using only non-zero values
    - Set minimum z-score to min_z (default -5)
    - Set all zero expressions to min_z (default -5)
    
    Args:
        values: Array-like input values
        mean: Optional pre-computed mean
        std: Optional pre-computed standard deviation
        min_z: Minimum z-score (default -5)
    Returns:
        numpy array of z-scores
    """
    # Convert to numpy array and identify non-zero values
    values = np.array(values)
    non_zero_mask = values >= low_exp
    non_zero_values = values[non_zero_mask]
    
    # If no non-zero values, return array of -5
    if len(non_zero_values) == 0:
        return np.full_like(values, min_z)
    
    # Calculate mean and std from non-zero values if not provided
    if mean != mean:  # Check for nan
        mean = np.mean(non_zero_values)
    if std != std:    # Check for nan
        std = np.std(non_zero_values)
        # Handle case where std is 0
        if std == 0:
            std = 1
    
    # Calculate z-scores
    zscores = np.full_like(values, min_z)  # Initialize with -5
    non_zero_zscores = (non_zero_values - mean) / std
    
    # Apply minimum threshold 
    non_zero_zscores = np.maximum(non_zero_zscores, min_z)
    
    # Put non-zero z-scores back in original positions
    zscores[non_zero_mask] = non_zero_zscores
    
    return zscores

def Z1Conversion(ExpMat, outname="test.z1.mat"):
    Z_mat = []
    for g, row in ExpMat.iterrows():
        tmp = ZscoreConverting(row.values)
        Z_mat.append(tmp)
    Z_mat = np.array(Z_mat)
    CT_Z1_DF = pd.DataFrame(data=Z_mat, index=ExpMat.index.values, 
                            columns=ExpMat.columns.values)
    CT_Z1_DF.to_csv(outname)
    return CT_Z1_DF

def Z1Conversion_V2(ExpMat, outname="test.z1.mat", low_exp = 0, min_z=-5):
    """
    Convert expression matrix to z-scores with zero handling
    
    Args:
        ExpMat: pandas DataFrame with genes as rows and cell types as columns
        outname: output file name for saving results
        
    Returns:
        pandas DataFrame with z-scores
    """
    Z_mat = []
    for g, row in ExpMat.iterrows():
        tmp = ZscoreConverting_V2(row.values, min_z=min_z)
        Z_mat.append(tmp)
    
    Z_mat = np.array(Z_mat)
    CT_Z1_DF = pd.DataFrame(data=Z_mat, 
                           index=ExpMat.index.values,
                           columns=ExpMat.columns.values)
    
    CT_Z1_DF.to_csv(outname)
    return CT_Z1_DF


In [ ]:
# Check how 0-expressed genes specificity score 
HumanCT_Z1 = Z1Conversion_V2(Subclass_Exp_DF, min_z=-5)
max_Z, min_Z = 5, -5
HumanCT_Z1 = HumanCT_Z1.clip(upper=max_Z, lower=min_Z)
HumanCT_Z1.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/subclass_MeanLogUMI.Z1.clip5.csv")

In [ ]:
# Check how 0-expressed genes specificity score 
HumanCT_Z1 = Z1Conversion_V2(Subclass_Exp_DF, min_z=-5)
max_Z, min_Z = 3, -3
HumanCT_Z1 = HumanCT_Z1.clip(upper=max_Z, lower=min_Z)
HumanCT_Z1.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/subclass_MeanLogUMI.Z1.clip3.csv")

In [ ]:
Cluster_Exp_DF = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/cluster_MeanLogUMI.csv", index_col=0)

In [ ]:
MouseCluster_CT_Z1 = Z1Conversion_V2(Cluster_Exp_DF, min_z=-5)
max_Z, min_Z = 5, -5
MouseCluster_CT_Z1 = MouseCluster_CT_Z1.clip(upper=max_Z, lower=min_Z)
MouseCluster_CT_Z1.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/cluster_MeanLogUMI.Z1V2.clip5.csv")

In [ ]:
MouseCluster_CT_Z1 = Z1Conversion_V2(Cluster_Exp_DF, min_z=-3)
max_Z, min_Z = 3, -3
MouseCluster_CT_Z1 = MouseCluster_CT_Z1.clip(upper=max_Z, lower=min_Z)
MouseCluster_CT_Z1.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/cluster_MeanLogUMI.Z1V2.clip3.csv")

In [ ]:
Subclass_Exp_DF = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/subclass_MeanLogUMI.csv", index_col=0)

In [ ]:
col_sums = Subclass_Exp_DF.sum(axis=0)
sum_mean = col_sums.mean()

# Normalize each column to have the same sum (1000 * mean)
target_sum = 1000 * sum_mean
normalized_expr = Subclass_Exp_DF.div(col_sums, axis=1) * target_sum

In [ ]:
# Calculate total expression sum across cell types for each gene
total_exp_per_gene = normalized_expr.sum(axis=1)

# Divide each expression value by the total for that gene to get fraction
specificity_scores = normalized_expr.div(total_exp_per_gene, axis=0)
specificity_scores.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/subclass_MeanLogUMI.specificity.csv")